### Socket

socket是进程间通信的一种方式，它的特点是可以实现不同主机间的进程通信。

```python
import socket

socket.socket(AddressFamily, Type)
```

- `AddressFamily`: 可以选择 `AF_INET` (IPv4) 或者 `AF_INET6` (IPv6)
- `Type`: socket类型，可以是 `SOCK_STREAM`用于TCP协议， 或者 `SOCK_DGRAM`用于UDP协议

#### 创建 socket
```python
import socket

# 创建tcp socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 创建udp socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# 不用的时候释放资源
s.close()
```


In [ ]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# 绑定本地的ip和端口号
# s.bind(('127.0.0.1', '4100'))

# ip地址不用写，会使用当前主机的ip
s.bind(('', 4100))

# python3中通过socket发送数据时要求使用bytes类型的数据
s.sendto('hello world!'.encode('utf-8'), ('127.0.0.1', 8080))

recv_msg = s.recvfrom(1024) # 1024表示本次接收消息的最大字节数

print(recv_msg[0].decode('utf-8'))

s.close()

### 多线程socket服务器

In [1]:
import socket
import threading
import time
from typing import Any

HOST = socket.gethostbyname(socket.gethostname())
PORT = 8000


def handle_request(client_sock: socket.socket, client_addr: Any):
    print(f'Connected to {client_addr}')
    msg = client_sock.recv(4096)
    if msg == "":
        return
    msg = msg.decode('utf-8')
    print(f'Received: {msg}')

    data = "HTTP/1.1 200 OK\r\n"
    data += "Content-Type: text/html; charset=utf-8\r\n"
    data += "\r\n"
    data += "<html><body>Hello</body></html>\r\n\r\n"
    time.sleep(2)
    client_sock.sendall(data.encode('utf-8'))
    client_sock.close()
    # client_socket.shutdown()
    print(f'Disconnected from {client_addr}')


def main():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    try:
        server.bind((HOST, PORT))
        # 允许最多5个connection,后续的conn会被拒绝
        server.listen()
        print(f"listening to {HOST}:{PORT}")
        while True:
            conn = server.accept()
            t = threading.Thread(target=handle_request, args=conn)
            t.start()

    except KeyboardInterrupt:
        print("shutting down...\n")
    except Exception as e:
        print("Error:\n")
        print(e)
    finally:
        server.close()

### socket使用上下文管理器

`socket.socket()`可以使用上下文管理器`with` 因为socket()返回的对象实现了 `__enter__` 和 `__exit__` 方法